# <a name="how"></a> Как сделать пулл реквест?

0. Выбираем, где хотим провести изменения, в форке репозитория (более предпочтительно, но не принципиально) или в самом репозитории (в этом случае нужно запросить у меня доступ).
1. Нужно произвести все желаемые изменения в семинарском ноутбуке. И убедиться, что эти изменения сохранены (юпитер у меня иногда тупит, поэтому жму трижды `ctrl-s` с интервалом около секунды).
  <br> Постарайтесь ограничиться минимальными изменениями. Так же убедитесь, что у вас актуальная версия репозитория. (Я сам в недавние ноутбуки могу теоретически каждый день коммитить, а мои изменения затирать не надо :) ).
2. Далее в этом ноутбуке (он умеет правильно генерировать `.md` файлы):
  <br>A.  <a href="#what" style="color:#856024"> Здесь </a> выбираем семинар(ы), к которому сделали правку. `../tools` выбирать не надо.
  <br>B.  <a href="#github" style="color:#856024"> Здесь </a> можно написать свой commit message, если есть желание. Можно оставить как есть. В этом репозитории нет культуры хороших сообщений к коммитам :)
  <br>C.  Запускаем этот ноутбук, он сгенерит `.md`-шки и закоммитит изменения на гитхаб.
3. Если изменение было в форке, то делаем пулл реквест.

### <a name="what"></a> Выбираем что коммитить

In [1]:
import glob
import os
import subprocess

highlevel_dirs = sum([
    #["../tools"], 
    sorted(glob.glob("../sem03*")),
    #sorted(glob.glob("../extra*")),
], [])

print("Highlevel dirs:", highlevel_dirs)

Highlevel dirs: ['../sem03-cmake-python-bindings']


In [2]:
tmp_dir = "./tmp_dir"
get_ipython().system('rm -r {tmp_dir} ; mkdir {tmp_dir} 2>&1 | grep -v "File exists"'.format(tmp_dir=tmp_dir))

### Генерируем все .md-шки стандартными средствами
\+ Делаем .md-шки очищенные для вывода. По этим .md-шкам можно будет смотреть реальную историю изменений. И дифф при пулреквестах.

In [3]:
def execute_all_in_parallel(tasks):
    ps = []
    for t in tasks:
        ps.append(subprocess.Popen(["bash", "-c", t], stdout=subprocess.PIPE, stderr=subprocess.PIPE))
    for p in ps:
        out, err = p.communicate()
        print(out.decode(), err.decode())

In [4]:
from multiprocessing import Pool

tasks = []

def convert_tasks(n, d):
    no_output_file = d + "_no_output"
    src_copy = str(abs(hash(n))) + '_' + os.path.basename(n)
    path = os.path.dirname(n)
    return [
        "jupyter nbconvert {} --to markdown --output {}".format(n, d),
        " && ".join([
            "cp {src} {tmp_dir}/{src_copy}",
            "jupyter nbconvert {tmp_dir}/{src_copy} --ClearOutputPreprocessor.enabled=True --inplace",
            "jupyter nbconvert {tmp_dir}/{src_copy} --to markdown --output {src_copy}",
            "cp {tmp_dir}/{src_copy}.md {path}/{no_output_file}.md",
        ]).format(src=n, no_output_file=no_output_file, dst=d, tmp_dir=tmp_dir, src_copy=src_copy, path=path),
    ]
    
for subdir in highlevel_dirs:
    notebooks = glob.glob(subdir + "/*.ipynb")
    print(subdir, notebooks)
    for m in glob.glob(subdir + "/*.md"):
        os.remove(m)
    if len(notebooks) == 1:
        tasks.extend(convert_tasks(notebooks[0], "README"))
    else:
        for n in notebooks:
            tasks.extend(convert_tasks(n, os.path.basename(n.replace(".ipynb", ""))))
        nmds = [os.path.basename(n).replace(".ipynb", ".md") for n in notebooks]
        with open(os.path.join(subdir, "README.md"), "w") as f:
            f.write('\n'.join(
                ['# Ноутбуки семинара'] + 
                ['* [{nmd}]({nmd})'.format(nmd=nmd) for nmd in nmds] + 
                ['']
            ))

print("\n".join(tasks))

execute_all_in_parallel(tasks)

../sem03-cmake-python-bindings ['../sem03-cmake-python-bindings/bindings.ipynb']
jupyter nbconvert ../sem03-cmake-python-bindings/bindings.ipynb --to markdown --output README
cp ../sem03-cmake-python-bindings/bindings.ipynb ./tmp_dir/2378846515383418137_bindings.ipynb && jupyter nbconvert ./tmp_dir/2378846515383418137_bindings.ipynb --ClearOutputPreprocessor.enabled=True --inplace && jupyter nbconvert ./tmp_dir/2378846515383418137_bindings.ipynb --to markdown --output 2378846515383418137_bindings.ipynb && cp ./tmp_dir/2378846515383418137_bindings.ipynb.md ../sem03-cmake-python-bindings/README_no_output.md
 [NbConvertApp] Converting notebook ../sem03-cmake-python-bindings/bindings.ipynb to markdown
[NbConvertApp] Writing 45540 bytes to ../sem03-cmake-python-bindings/README.md

 [NbConvertApp] Converting notebook ./tmp_dir/2378846515383418137_bindings.ipynb to notebook
[NbConvertApp] Writing 49100 bytes to ./tmp_dir/2378846515383418137_bindings.ipynb
[NbConvertApp] Converting notebook ./

### Магические исправления

Стандартная конвертилка не учитывает некоторых особенностей маркдауна на гитхабе и некоторых особенностей моих ноутбуков. Поэтому результат надо допилить напильником.

In [5]:
import re


def basic_improve(fname):
    with open(fname, "r") as f:
        r = f.read()
    for b in ["\x00", "\x1B", "\x08"]:
        r = r.replace(b, "")
    with open(fname, "w") as f:
        f.write(r)
    return "dos2unix {}".format(fname)

def improve_md(fname):
    with open(fname, "r") as f:
        r = f.read()
    r = r.replace("```python\n%%cpp", "```cpp\n%%cpp")
    r = r.replace("```python\n%%cmake", "```cmake\n%%cmake")
    r = r.replace('\n', "SUPER_SLASH" + "_N_REPLACER")
    
    r = re.sub(r'\<\!--MD_BEGIN_FILTER--\>.*?\<\!--MD_END_FILTER--\>', "", r)
    
    r = r.replace("SUPER_SLASH" + "_N_REPLACER", '\n')
    
    template = "#""MAGICS_SETUP_END"
    bpos = r.rfind(template)
    if bpos != -1:
        r = r[bpos + len(template):]
        template = "```"
        bpos = r.find(template)
        assert bpos >= 0
        r = r[bpos + len(template):]
    
    
    template = "<""!-- MAGICS_SETUP_PRINTING_END -->"
    bpos = r.rfind(template)
    if bpos != -1:
        r = r[bpos + len(template):]
    
    def file_repl(matchobj, path=os.path.dirname(fname)):
        fname = os.path.join(path, matchobj.group(1))
        if fname.find("__FILE__") == -1:
            with open(fname, "r") as f:
                return "\n<pre>\n" + f.read() + "\n</pre>\n"
    
    r = r.replace("</td>", "")
    r = r.replace("</tr>", "")
    
    r = re.sub(r'\<\!--MD_FROM_FILE (.*?) --\>', file_repl, r)
    with open(fname, "w") as f:
        f.write(r)
        
def improve_file(fname):
    if fname.endswith(".md"):
        improve_md(fname)


In [6]:
tasks = []
shell_tasks = []


for sfx in [".ipynb", ".md"]:
    for hdir in highlevel_dirs:
        for fname in glob.glob("./{}/*".format(hdir) + sfx):
            shell_tasks.append(basic_improve(fname))
            tasks.append(lambda fname=fname: improve_file(fname))
            
execute_all_in_parallel(shell_tasks)
for t in tasks:
    t()

 dos2unix: converting file ./../sem03-cmake-python-bindings/bindings.ipynb to Unix format...

 dos2unix: converting file ./../sem03-cmake-python-bindings/README.md to Unix format...

 dos2unix: converting file ./../sem03-cmake-python-bindings/README_no_output.md to Unix format...



### Смотрим изменения

In [7]:
for subdir in highlevel_dirs:
    get_ipython().system("git diff {}/*_no_output*".format(subdir))

### <a name="github"></a> Коммитим на github

In [8]:
cmds = []
add_cmd = "git add --ignore-errors "
add_cmd_f = "git add --ignore-errors -f "
for subdir in highlevel_dirs:
    for sfx in [".ipynb", ".md", ".c", ".cpp"]:
        cmds.append(add_cmd + " {}/*{}".format(subdir, sfx))
    cmds.append(add_cmd_f + " -f {}/bash_popen_tmp/*.html".format(subdir))
    cmds.append(add_cmd_f + " -f {}/interactive_launcher_tmp/*.log".format(subdir))
    cmds.append("git add -u {}".format(subdir))
    
def execute_cmd(cmd):
    print(">", cmd)
    get_ipython().system(cmd)
    
for cmd in cmds:
    execute_cmd(cmd)
# execute_cmd("git add -u")
execute_cmd("git commit -m 'yet another update'")
execute_cmd("git push origin master")

> git add --ignore-errors  ../sem03-cmake-python-bindings/*.ipynb
> git add --ignore-errors  ../sem03-cmake-python-bindings/*.md
> git add --ignore-errors  ../sem03-cmake-python-bindings/*.c
> git add --ignore-errors  ../sem03-cmake-python-bindings/*.cpp
> git add --ignore-errors -f  -f ../sem03-cmake-python-bindings/bash_popen_tmp/*.html
fatal: pathspec '../sem03-cmake-python-bindings/bash_popen_tmp/*.html' did not match any files
> git add --ignore-errors -f  -f ../sem03-cmake-python-bindings/interactive_launcher_tmp/*.log
fatal: pathspec '../sem03-cmake-python-bindings/interactive_launcher_tmp/*.log' did not match any files
> git add -u ../sem03-cmake-python-bindings
> git commit -m 'yet another update'
[master 5251720] yet another update
 9 files changed, 9113 insertions(+)
 create mode 100644 sem03-cmake-python-bindings/README.md
 create mode 100644 sem03-cmake-python-bindings/README_no_output.md
 create mode 100644 sem03-cmake-python-bindings/bindings.ipynb
 create mode 100644 se

In [9]:
!git add ../README.md
!git commit -m "Update readme"
!git push origin master

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../sem01-intro-linux/interactive_launcher_tmp/104342371564797143.log
	modified:   save_them_all.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../extra-c-basics/001.in
	../sem01-intro-linux/2
	../sem01-intro-linux/a.sh
	../sem01-intro-linux/a.txt
	../sem01-intro-linux/b.txt
	../sem01-intro-linux/err
	../sem01-intro-linux/err.txt
	../sem01-intro-linux/file.txt
	../sem01-intro-linux/file2.txt
	../sem01-intro-linux/interactive_launcher_tmp/104342371564797143.log.md
	../sem01-intro-linux/interactive_launcher_tmp/134856167677051543.log.md
	../sem01-intro-linux/interactive_launcher_tmp/159458045408584490.log.md
	../sem01-intro-linux/interactive_launcher_tmp/160806476576838908.log.md
	../sem01-intro-linux/intera